In [11]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_columns', None)

In [12]:
datalocation = './datasets.uwf.edu/data/UWF-ZeekData22/'

cols = ['resp_pkts', 
        'service', 
        'orig_ip_bytes', 
        'local_resp', 
        'missed_bytes',
        'protocol', 
        'duration', 
        'conn_state', 
        'dest_ip', 
        'orig_pkts',
        'community_id', 
        'resp_ip_bytes', 
        'dest_port', 
        'orig_bytes',
        'local_orig', 
        'datetime', 
        'history', 
        'resp_bytes', 
        'uid', 
        'src_port',
        'ts', 
        'src_ip', 
        'mitre_attack_tactics'
        ]

df = pd.DataFrame(columns=cols)

for root, dirs, files in os.walk(datalocation):
    for name in files:
        if name.endswith('.csv'):
            dfi = pd.read_csv(os.path.join(root, name))       
        elif name.endswith('.parquet'):
            dfi = pd.read_parquet(os.path.join(root, name), engine='pyarrow')
            
        else:
          continue
        if dfi.shape[1] != 23:
          print("wrong shape: ", os.path.join(name))
          continue
        print(os.path.join(name))
        df =  pd.DataFrame(np.concatenate([df.values, dfi.values]), columns=cols)
print(df.shape)
        

part-00000-36d29ab0-aecd-4e34-8d7a-cf9b94be3fea-c000.csv
part-00000-0af89d10-df53-44fd-b124-a8a496fd5023-c000.csv
part-00000-8283e5f3-ac2e-4523-a7f0-f98f0c7d879f-c000.csv
part-00000-5b4f5c3f-e8a9-4020-8fa1-e8985f7c27f3-c000.csv
part-00000-bd55a170-4439-45b0-9122-9a6b9da9760e-c000.csv
part-00000-318611a1-7cdc-4dd0-9348-c6368917fd0c-c000.csv
part-00000-95e0a460-e7c5-4b35-8367-c2e6fbbcf9e1-c000.csv
part-00000-0f7bf56c-ed34-4235-a8d4-1c86eac7ff66-c000.csv
part-00000-15e3dd03-ea76-429e-a52a-ce96a90517f9-c000.csv
part-00000-12352875-21a6-4bd8-aaf1-922e121ef289-c000.csv
part-00000-26e9208e-7819-451b-b23f-2e47f6d1e834-c000.snappy.parquet
wrong shape:  part-00000-d32a9d5e-45b7-4e51-807e-1af297aba2df-c000.csv
part-00000-df678a79-4a73-452b-8e72-d624b2732f17-c000.snappy.parquet
part-00000-b1a9fc13-8068-4a5d-91b2-871438709e81-c000.snappy.parquet
part-00000-7c2e9adb-5430-4792-a42b-10ff5bbd46e8-c000.snappy.parquet
part-00000-3f86626a-1225-47f9-a5a2-0170b737e404-c000.snappy.parquet
part-00000-cbf26680

In [13]:
df["orig_bytes"].loc[df["orig_bytes"].notnull() == False] = -10000
edges = pd.IntervalIndex.from_breaks([float('-inf'), -1, 0, 78, 156, float('inf')])
df['orig_bytes'] = pd.cut(df['orig_bytes'].astype(float), edges)

In [14]:
df["orig_pkts"].loc[df["orig_pkts"].notnull() == False] = -10000
edges = pd.IntervalIndex.from_breaks([-1, 0, 1, 2, 3, 4, float('inf')])
df['orig_pkts'] = pd.cut(df['orig_pkts'].astype(float), edges)

/tmp/ipykernel_610/2626207123.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["orig_pkts"].loc[df["orig_pkts"].notnull() == False] = -10000


IP bins:
- Class A: First octet value 0–126;
- Class B: First octet value 128–191;
- Class C: First octet value 192–223;
- Class D: First octet value 224–239;
- Class E: First octet value 240–254.
- IPv6 local link (fe80:)
- IPv6 global internet (2001:, ff02:)

In [15]:
df['dest_ip_class'] = df['dest_ip'].str.split('.', expand=True).iloc[:, 0]
df['dest_ip_class'] = df['dest_ip_class'].str.split(':', expand=True).iloc[:, 0]
df['dest_ip_class'].replace('ff02', '2001', inplace=True)
df['dest_ip_class'].replace('fe80', '999', inplace=True)
edge0 = float('-inf')
edge1 = 127
edge2 = 191
edge3 = 223
edge4 = 239
edge5 = 255
edge6 = 1000
edge7 = float('inf')
edges = pd.IntervalIndex.from_arrays([edge0, edge1, edge2, edge3, edge4, edge5, edge6], [edge1, edge2, edge3, edge4, edge5, edge6, edge7])
df['dest_ip'] = pd.cut(df['dest_ip_class'].astype(float), edges)

In [16]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for i in ['service', 'protocol', 'history', 'mitre_attack_tactics', 'orig_pkts', 'orig_bytes', 'dest_ip']:
    df[i] = le.fit_transform(df[i])

In [17]:
#np.save('preprocessed_df', df)
df[['orig_bytes', 'orig_pkts', 'history', 'protocol', 'service', 'dest_ip', 'mitre_attack_tactics']].astype(str).to_parquet('preprocessed_df.parquet')

: 